In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/20news/instances.pkl' mode='rb' closefd=True>


In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(11258, 1995)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 0 0 

## run

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
  Time Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0  409  0  0   1126   1133  1135  0.01  0.33  0.38  0.79  0.71
1  486  1  0   1098   1105  1106  0.01  0.38  0.41  0.89  0.80
2  485  2  0   1078   1087  1086  0.01  0.37  0.40  0.87  0.77
3  508  3  0   1062   1074  1071  0.01  0.38  0.40  0.86  0.78
4  504  4  0   1048   1061  1057  0.01  0.38  0.40  0.85  0.77
5  510  5  0   1036   1051  1046  0.01  0.38  0.41  0.84  0.76
6  509  6  0   1026   1042  1036  0.01  0.38  0.41  0.82  0.75
7  502  7  0   1016   1032  1028  0.01  0.38  0.41  0.81  0.74
8  516  8  0   1007   1023  1020  0.01  0.38  0.42  0.80  0.74

Exception ignored in: <_io.FileIO name='model/20news/ncrp/20news-mncrp-alp0.10.050.01-eta10-gam0.01-epoch50/log' mode='wb' closefd=True>


 0 11258 594678.0 write one use get say article know go people like
   0-1 5322 165294.0 write one say people article god think make know get
     0-1-1 5322 15401.0 write people article say one think make god get go
   0-2 4471 125644.0 use get one file system write key work problem like
     0-2-1 4470 1411.0 colormap printer amp reference cycle compatible postscript double clock ibm
     0-2-2 1 0.0 bhj series ram dave knowledge prevent james throw washington legal
   0-3 1120 49425.0 game team play win year player go get write hockey
     0-3-1 1120 1434.0 gm flyers puck game goal penalty get winner guy team
   0-4 300 31305.0 space launch use satellite april year mission nasa orbit center
     0-4-1 300 849.0 professor palestinian japanese history university economic enforcement blood pittsburgh wiretap
   0-5 14 1108.0 rg oo de eus mr ma van mw mi mg
     0-5-1 14 1.0 pc bhj peace ram dave knowledge prevent james throw washington
   0-6 7 918.0 dos good excellent miss cover modul